## ---
# Unit11: Ambient Noise Tomography

This notebook has the activities of the Course **ProSeisSN**. It deals with time series processing using a passive seismic dataset using [ObsPy](https://docs.obspy.org/).

#### Dependencies: Obspy, Numpy, Matplotlib
#### Reset the Jupyter notebook in order to run it again, press:
***Kernel*** -> ***Restart & Clear Output***

In [ ]:
"""
====================== PyCWT: wavelet spectral analysis in Python ======================
"""
!pip install pycwt
import pycwt

In [ ]:
#------ Import Libraries
import sys
import os
    
#------ Work with the directory structure to include auxiliary codes
print('\n Local directory ==> ', os.getcwd())
print('  - Contents: ', os.listdir(), '\n')

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path+"/CodePy")

%run ../CodePy/ImpMod.ipynb

#------ Alter default matplotlib rcParams
from matplotlib import rcParams
import matplotlib.dates as dates
# Change the defaults of the runtime configuration settings in the global variable matplotlib.rcParams
plt.rcParams['figure.figsize'] = 9, 5
#plt.rcParams['lines.linewidth'] = 0.5
plt.rcParams["figure.subplot.hspace"] = (.9)
plt.rcParams['figure.dpi'] = 100
#------ Magic commands
%matplotlib inline
%matplotlib widget
#%pylab notebook
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

---
## Read data files from TTB22 as SEG2. Create a new stream from the SEG2 stream
- Read phone positions
- Select and read data

In [ ]:
"""
====================== READ PHONES LOCATIONS ======================
"""
#------ Read the phones cartesian locations
#--- Reads the CSV file with (x, y)m locations
ttb_loc = u.RGloc('../Data/'+'ttb_loc.dat')
#------ Read the phones geographic locations
#--- Reads the CSV file with (lat,lon) in degress locations
ttb_gloc = u.RGloc('../Data/'+'ttb_gloc.dat')
#
#------ Plot gather in cartesian
p.pgather(ttb_loc[:,1], ttb_loc[:,2], ttb_loc[:,0], coord='cartesian')

In [ ]:
"""
====================== READ THE SEISMIC DATA LOCALLY ======================
File hints:
3710 and 3720 -> several events
3740 -> 2 events
3790 -> 1 event (6-9)s
"""
#------ Read the seismic data
ent = str(np.random.choice(np.arange(3700, 3811, 10)))
ent = input(f'   Enter a file number in [3695, 3810], rtn=random:\n') or ent
ent = ent.rstrip().split(' ')
print(f">> Read with data file {ent}")
ent = '../Data/ttb/'+ent[0]+'.dat'
#
#------- Read the data file as a SEG2 object.
st     = read(ent)
#
#------- Print stream information
dummy = float(st[-1].stats.seg2.RECEIVER_LOCATION)
print(f">> Gather acquired on {st[0].stats.starttime}, has {int(st[0].stats.npts)} data points.")
"""
================= Create a new stream from the SEG2 stream ======================
                         Retain a gather copy
"""
#------ Create a new stream from the SEG2 stream.
#       1) Adds coordinates to gather. Stores a copy in gather0
#       2) Gather baricenter = bcenter.
gather, bcenter = u.creastrm(st, ttb_gloc)
gather0 = gather.copy()
#
#--- Phone choice
phone = None


---
## Data processing
- Filter data
- Display the seismogram
### Filter and look at seismogram and to the frequency content

In [ ]:
#
"""
================= Filter data and look at the frequency contents ======================
                    Create a new stream from the SEG2 stream
"""
#
#------- Remove mean and trend + filter the stream
#--- Filter parameters: change them as you wish.
MTparam = [ 1,   1,    'bp',  10.,   40.,   0,    0]
# └─────> [dtr, line, ftype, Fmin, Fmax, taper, gain]
#                                          └─> data will be windowed at trace normalization and spectral whitening
ent = str(MTparam[3]) + ' ' + str(MTparam[4])
ent = input(f'\n Enter filter min and max frequencies (dflt = {MTparam[3]}, {MTparam[4]})') or ent
ent = ent.rstrip().split(' ')
MTparam[3], MTparam[4] = [float(dummy) for dummy in ent]
#
gather = u.otrstr(gather, MTparam)
#
#------- Check frequency contents to accept preprocessing
#--- Pick up a random phone/trace
phone = phone if phone is not None else np.random.randint(1, len(gather)+1)
print(f' Random phone {phone} ')
#--- Go to trace instead of phone: trace = phone -1
phone = phone - 1
#--- Relative time: nummpy array
time = gather[phone].times(type="relative")
#--- Plot Trace+Spectrogram
p.Pspect(time, gather[phone])
#
#------- Once filtering is accepted create a new backup for gather
ent = input(f' Run this cell again (rtn= No, else plot Spectrogram)?: ') or False
if not ent:
    gather0 = gather.copy()
    print(f' A new stream backup was created.')
else:
    gather = gather0.copy()

### Down-sample the data
- Down-sample the data to number of pints compatible with the upper limit of the bandpass filter
- Reduce computational costs

In [ ]:
#
gather = gather0.copy()
"""
================= Downsample stream by an integer factor ======================
"""
print(f'\n>> Phone {phone+1} has {gather[phone].stats.npts} data points with a sampling rate of {gather[phone].stats.sampling_rate}Hz,')
dummy =  u.divisors(int(gather[phone].stats.sampling_rate), MTparam[4])
print(f'    this sampling rate can be lowered to the following integer values {dummy}Hz')
ent = input(f'\n<< Enter a new sampling rate from the above list:')
ent = float( ent.rstrip().split(' ')[0] )
"""
Decimate (other possiblity is resample)
1) Only every decimation_factor-th sample remains in the trace.
2) Prior to decimation it is applyed a lowpass filter to prevente aliasing artifacts.
3) To abort when
           len(data) % decimation_factor != 0
    set strict_length=True.
"""
#--- // is a floor division = integer floor. Sanity
factor = int(gather[phone].stats.sampling_rate / ent)
if gather[phone].stats.npts % factor != 0: raise ValueError("Decimation factor is not an integer.")
gather.decimate(factor=factor, strict_length=True)
#--- Check on Fmax
MTparam[4] = MTparam[4] if MTparam[4] <= ent else ent
#
print(f'\n>> Phone {phone+1} has now {gather[phone].stats.npts} data points with a new sampling rate of {gather[phone].stats.sampling_rate}Hz.')
print(f'    Resampled data is FIR low-pass filtered to prevent aliasing, with a decimation factor of  {factor}.')
#
#------- Check frequency contents of a trace to accept downsampling
#--- Relative time: nummpy array
time = gather[phone].times(type="relative")
#--- Plot Trace+Spectrogram
p.Pspect(time, gather[phone])
#
#------- Once filtering is accepted create a new backup for gather
ent = input(f' Run this cell again (rtn= No)?: ') or False
if not ent:
    gather0 = gather.copy()
    print(f' A new stream backup was created.')
else:
    gather = gather0.copy()

In [ ]:
"""
====================== Plot Seismogram ======================
"""
gather.plot(type='section',
            scale=1.3, alpha=.7,
            orientation='horizontal')

In [ ]:
"""
====================== Zoom in to select [t0, t1]======================
"""
#------ Zoom in the seismogram

ent = input(f' Enter t0 and t1 to zoom: ')

ent = ent.rstrip().split(' ')
t0 = float(ent[0])
t1 = float(ent[1])
#
dt = gather[0].stats.starttime
gather.plot(type='section',
            scale=1.3, alpha=.7,
            starttime=dt+t0, endtime=dt+t1,
            orientation='horizontal')

In [ ]:
"""
====================== BEAMFORMING ======================
"""
dummy = UTCDateTime()
# ---------- Use FK Analysis
out, stime, etime = u.BeamFK(gather, [MTparam[3], MTparam[4]], ttb_gloc)
print("\n>> Total time in Beamforming: %f\n" % (UTCDateTime() - dummy))
#---------- Change output
t, rel_power, abs_power, baz, slow = out.T
#--- Time
T = np.linspace(stime, etime, num=len(out))
#--- Semblance -> Fisher
F = (len(gather)-1) * out[:, 1] / (1 - out[:, 1])
#--- FK power
FKp = out[:, 2]
#--- baz
#out[:, 3] = out[:, 3] % 360.
baz = baz % 360.
#--- Slowness -> Velocity
V = 1.e3 / out[:,4]
#------------- print
sys.stdout.write('\n')
print(f'\n>>  t   Fisher  FKpwr   baz(deg) vel(m/s)')    
for i in range(len(out)):
    print(f'   {round(T[i],2)}, {round(F[i],2)}, {round(FKp[i],2)}, {round(baz[i],2)}, {round(V[i],2)}')    
#------------ Plot
p.pltbaz(out, stime , etime)
"""
1) CLICK ON BLUE BAR TO EXPAND
"""

#### Is this velocity expected to be real?

<div style="text-align: center;">
<img src="./rockVel.png" width="400">
</div>

### Choose phone pairs

<div style="text-align: center;">
<img src="./ttbg.png" width="500">
</div>

In [ ]:
"""
====================== Local functions ======================
"""
#
# ---------- Cuts data into time segments ----------
"""
Cuts continous noise data into user-defined segments, estimate statistics for each segment and keep timestamps for later use.
  <Args>
    tr        -> A trace, an 1-dimensional timeseries array
    step      -> % overlapping (0, 1) between two sliding windows
    cc_len    -> Segment length (sec) to cut trace data
  <Returns>
    trace_stdS: standard deviation of the noise amplitude of each segment
    trace_madS: mad of the noise amplitude of each segment
    dataS_t:    timestamps of each segment
    dataS:      2D matrix of the segmented data
"""
def cut_trace(tr, cc_len, step):
#
#------ Trace stats and initialize return variables
#--- sampling_rate
    sps       = int(tr.stats.sampling_rate)
#--- trace window length (sec)
    tw        = round(tr.stats.endtime - tr.stats.starttime, 0)
#--- Date and time of the first data sample given in UTC (default value is “1970-01-01T00:00:00.0Z”)
    starttime = tr.stats.starttime - obspy.UTCDateTime(1970,1,1)
#--- Overlap in sec
    step = cc_len * step
#--- Number of segments and of points in each segment
    nseg      = int(np.floor((tw - cc_len)/step))    #tw/24*86400-cc_len
    npts = int(cc_len * sps)
#--- Copy data into array and check if data is shorter than the trim
    data      = tr.data
    if data.size < sps * nseg: raise ValueError("cc_len must be << trace.")
#--- Overlapping (sec) between two sliding windows
    step      = np.ceil(cc_len * step).astype(int)
#
#------ initialize variables
    dataS_t = []; dataS = []
#--- Relevant arrays
    dataS      = np.zeros(shape=(nseg, npts),dtype=np.float32)
    trace_madS = np.zeros(nseg,dtype=np.float32)
    trace_stdS = np.zeros(nseg,dtype=np.float32)
    trace_madS = np.zeros(nseg,dtype=np.float32)
    dataS_t    = np.zeros(nseg,dtype=np.float32)
#
#------ Statistic to detect segments that may be associated with a given event
#--- median absolute deviation over all trace
    all_madS = np.median(np.absolute(data - np.median(data)))
#--- standard deviation over all noise window
    all_stdS = np.std(data)
    if all_madS==0 or all_stdS==0 or np.isnan(all_madS) or np.isnan(all_stdS):
        print("Warn: madS or stdS == 0 for %s" % tr)
        return None, dataS_t, dataS
#

#    tw, cc_len, step 60.0 3.0 1.5
#    data.size, sps, nseg, npts 3000 50 38 150
    print('tw, cc_len, step', tw, cc_len, step)
    
    indx1 = 0
    for iseg in range(nseg):
        indx2 = indx1+npts

        
        print('data, iseg, indx1,indx2', data[indx1:indx2].size, iseg, indx1, indx2)

        
        dataS[iseg, :] = data[indx1:indx2]
        trace_madS[iseg] = (np.max(np.abs(dataS[iseg]))/all_madS)
        trace_stdS[iseg] = (np.max(np.abs(dataS[iseg]))/all_stdS)
        dataS_t[iseg]    = starttime+step*iseg
        indx1 = indx1+step*sps
#
#------ Data conditioning. It is assumed data is already demeaned and detrended
    dataS = taper(dataS)
#
#------  Returns
    return trace_stdS, trace_madS, dataS_t, dataS
#
# -------------- End of function   ---------------------
#
# ---------- Applies taper to time segments ----------
"""
Applies a cosine taper using obspy functions
  <Args>
    data -> An input data matrix
  <Returns>
    data -> A tapered data matrix
"""
def taper(data):
#ndata = np.zeros(shape=data.shape,dtype=data.dtype)
    if data.ndim == 1:
        npts = data.shape[0]
        # window length
        if npts*0.05>20:wlen = 20
        else:wlen = npts*0.05
        # taper values
        func = _get_function_from_entry_point('taper', 'hann')
        if 2*wlen == npts:
            taper_sides = func(2*wlen)
        else:
            taper_sides = func(2*wlen+1)
        # taper window
        win  = np.hstack((taper_sides[:wlen], np.ones(npts-2*wlen),taper_sides[len(taper_sides) - wlen:]))
        data *= win
    elif data.ndim == 2:
        npts = data.shape[1]
        # window length
        if npts*0.05>20:wlen = 20
        else:wlen = npts*0.05
        # taper values
        func = _get_function_from_entry_point('taper', 'hann')
        if 2*wlen == npts:
            taper_sides = func(2*wlen)
        else:
            taper_sides = func(2*wlen + 1)
        # taper window
        win  = np.hstack((taper_sides[:wlen], np.ones(npts-2*wlen),taper_sides[len(taper_sides) - wlen:]))
        for ii in range(data.shape[0]):
            data[ii] *= win
#
#------  Returns
    return data
#
# -------------- End of function   ---------------------
#
# ---------- Spectral spectral normalization and whitening ----------
"""
Transforms to frequency domain, whitens the amplitude spectrum in the frequency domain between *freqmin* and *freqmax*,
 and returns the whitened fft.
  <Args>
    tr: A trace: 1-dimensional timeseries array
    dt: The sampling space of the `data`
    freqmin: The lower frequency bound
    freqmax: The upper frequency bound
    smooth_N: integer, it defines the half window length to smooth
    freq_norm: whitening method between 'one-bit' and 'RMA'
  <Returns>
    FFTRawSign: The FFT (numpy.ndarray) of the whitened input trace in [freqmin, freqmax]
"""
#
def whiten(data, delta, freqmin, freqmax, smooth_N, freq_norm):
#
#------ Speed up FFT by padding to optimal size for FFTPACK
    if data.ndim == 1:
        axis = 0
    elif data.ndim == 2:
        axis = 1
#
    Nfft = int(next_fast_len(int(data.shape[axis])))
#
#------ Apodization number to the left and to the right
    Napod = 100
#
    Nfft = int(Nfft)
    freqVec = scipy.fftpack.fftfreq(Nfft, d=delta)[:Nfft // 2]
    J = np.where((freqVec >= freqmin) & (freqVec <= freqmax))[0]
    low = J[0] - Napod
    if low <= 0:
        low = 1
#
    left = J[0]
    right = J[-1]
    high = J[-1] + Napod
    if high > Nfft/2:
        high = int(Nfft//2)
#
    FFTRawSign = scipy.fftpack.fft(data, Nfft,axis=axis)
#
#------  Left tapering:
    if axis == 1:
        FFTRawSign[:,0:low] *= 0
        FFTRawSign[:,low:left] = np.cos(
            np.linspace(np.pi / 2., np.pi, left - low)) ** 2 * np.exp(
            1j * np.angle(FFTRawSign[:,low:left]))
#
#------ Pass band:
        if freq_norm == 'phase_only':
            FFTRawSign[:,left:right] = np.exp(1j * np.angle(FFTRawSign[:,left:right]))
        elif freq_norm == 'rma':
            for ii in range(data.shape[0]):
                tave = moving_ave(np.abs(FFTRawSign[ii,left:right]),smooth_N)
                FFTRawSign[ii,left:right] = FFTRawSign[ii,left:right]/tave
#
#------  Right tapering:
        FFTRawSign[:,right:high] = np.cos(
            np.linspace(0., np.pi / 2., high - right)) ** 2 * np.exp(
            1j * np.angle(FFTRawSign[:,right:high]))
        FFTRawSign[:,high:Nfft//2] *= 0
#
#------  Hermitian symmetry -> input is real
        FFTRawSign[:,-(Nfft//2)+1:] = np.flip(np.conj(FFTRawSign[:,1:(Nfft//2)]),axis=axis)
    else:
        FFTRawSign[0:low] *= 0
        FFTRawSign[low:left] = np.cos(
            np.linspace(np.pi / 2., np.pi, left - low)) ** 2 * np.exp(
            1j * np.angle(FFTRawSign[low:left]))
#
#------  Pass band:
        if freq_norm == 'phase_only':
            FFTRawSign[left:right] = np.exp(1j * np.angle(FFTRawSign[left:right]))
        elif freq_norm == 'rma':
            tave = moving_ave(np.abs(FFTRawSign[left:right]),smooth_N)
            FFTRawSign[left:right] = FFTRawSign[left:right]/tave
#
#------  Right tapering:
        FFTRawSign[right:high] = np.cos(
            np.linspace(0., np.pi / 2., high - right)) ** 2 * np.exp(
            1j * np.angle(FFTRawSign[right:high]))
        FFTRawSign[high:Nfft//2] *= 0
#
#------  Hermitian symmetry  -> input is real
        FFTRawSign[-(Nfft//2)+1:] = FFTRawSign[1:(Nfft//2)].conjugate()[::-1]
#
#------  Returns
    return FFTRawSign
#
# -------------- End of function   ---------------------
#
# ---------- Wrapper for spectral normalization and whitening ----------
"""
Transforms to frequency domain, whitens the amplitude spectrum in the frequency domain between *freqmin* and *freqmax*,
 and returns the whitened fft.
  <Args>
    data: A 2D matrix of all segmented noise data
    dt: The sampling distance in seconds
    freqmin:   The lower frequency bound
    freqmax:   The upper frequency bound
    smooth_N:  Integer, it defines the half window length to smooth
    time_norm: Time-domain normalization -> 'one_bit' or 'rma'
    freq_norm: Whitening method -> 'one-bit' and 'RMA'
  <Returns>
    FFTRawSign: The FFT (numpy.ndarray) of the whitened input trace in [freqmin, freqmax]
"""
def noise_processing(data, delta, freqmin, freqmax, 
                     smooth_N = 100, time_norm = False, freq_norm = False):
#
    N = data.shape[0]
#
#------  Time normalization
    if time_norm:
#--- Sign normalization
        if time_norm == 'one_bit':
            white = np.sign(data)
#--- Normalization over smoothed absolute average; running mean
        elif time_norm == 'rma':
            white = np.zeros(shape=data.shape,dtype=data.dtype)
            for kkk in range(N):
                white[kkk,:] = data[kkk,:]/moving_ave(np.abs(data[kkk,:]),smooth_N)
#--- Don't normalize
    else:
        white = data
#
#------ whiten
    if freq_norm:
#--- Whiten and return FFT
        White = whiten(white, delta, freqmin, freqmax, smooth_N, freq_norm)
    else:
#--- Return FFT
        Nfft = int(next_fast_len(int(dataS.shape[1])))
        White = scipy.fftpack.fft(white, Nfft, axis=1)
#
#------  Returns
    return White
#
# -------------- End of function   ---------------------
#
# ---------- Running smooth average ----------
"""
This function does running smooth average for an array (use numba for performance)
  <Args>
    A: 1-D array of data to be smoothed
    N: Defines the half window length to smooth (integer)
  <Returns>
    B: 1-D array with smoothed data
"""
def moving_ave(A,N):
#
    A = np.concatenate((A[:N],A,A[-N:]),axis=0)
    B = np.zeros(A.shape,A.dtype)

    tmp=0.
    for pos in range(N,A.size-N):
        # do summing only once
        if pos==N:
            for i in range(-N,N+1):
                tmp+=A[pos+i]
        else:
            tmp=tmp-A[pos-N-1]+A[pos+N]
        B[pos]=tmp/(2*N+1)
        if B[pos]==0:
            B[pos]=1
    return B[N:-N]
#
# -------------- End of function   ---------------------
#
# ---------- Cross-correlation ----------
"""
Does the cross-correlation in freq domain. Keeps the sub-stacks of the cross-correlation if needed, taking advantage of
 the linear relationship of ifft. Stacking is performed in spectrum domain, reducing the total number of ifft.
 
  <Args>
    fft1:          Source station power (smoothed) spectral density
    fft2:          Receiver station raw FFT spectrum
    dt:            sampling rate (in s)
    freqmin:       minimum frequency (Hz)
    freqmax:       maximum frequency (Hz)
    Nfft:          number of frequency points for ifft
    maxlag:        maximum lags to keep in the cross correlation
    dataS_t:       matrix of datetime object
    method:        'xcorr' or "coherency"
    substack:      sub-stack cross-correlationS or not
    substack_len:  multiples of cc_len to stack ove
    smoothspect_N: number of points to be smoothed for running-mean average (freq-domain, method=="coherency")
  <Returns>
    s_corr: 1D or 2D matrix of the averaged or sub-stacks of cross-correlation functions in time domain
    t_corr: timestamp for each sub-stack or averaged function
    n_corr: number of included segments for each sub-stack or averaged function
"""
#

def correlate(fft1, fft2, dt, dataS_t,
              freqmin, freqmax, maxlag,
              method = 'xcorr', substack = False, substack_len  = None, smoothspect_N = None):
#
#------ check on fft1/2 dimensions: put a cap on them, assuning same diensions in both
    if len(fft1) != len(fft2): raise ValueError("fft1 and fft2 must have the same length")
#--- nwin = number of segments in the 2D fft(i) matrix
#
#------ fft1 & fft2 are 2-D
    if fft1.ndim == 2:
        nwin  = fft1.shape[0]
        Nfft = fft1.shape[1]
        Nfft2 = Nfft//2
#--- Cap second dimension to Nfft2
        fft1 = fft1[:, :Nfft2]
        fft2 = fft2[:, :Nfft2]
#--- convert 2D array into 1D
        corr = np.zeros(nwin*Nfft2,dtype=np.complex64)
#        corr = fft1.reshape(fft1.size,)*fft2.reshape(fft2.size,)
        corr = fft1.reshape(-1) * fft2.reshape(-1)
#
#------ fft1 & fft2 are 1-D
    elif fft1.ndim == 1:
        nwin  = 1
        Nfft = fft1.shape[0]
        Nfft2 = Nfft//2
#--- Cap to Nfft2
        fft1 = fft1[:Nfft2]
        fft2 = fft2[:Nfft2]
        corr = np.zeros(nwin*Nfft2,dtype=np.complex64)
        corr = fft1 * fft2
    else:
        raise ValueError("fft must be either 1 or 2-dimensions")
#
#------ method == coherency
    if method == "coherency":
        temp = moving_ave(np.abs(fft2.reshape(fft2.size,)),smoothspect_N)
        corr /= temp
    corr  = corr.reshape(nwin,Nfft2)            #it was Nfft2
#
#------ method != coherency
    if substack:
        if substack_len == cc_len:
            # choose to keep all fft data for a day
            s_corr = np.zeros(shape=(nwin,Nfft),dtype=np.float32)   # stacked correlation
            ampmax = np.zeros(nwin,dtype=np.float32)
            n_corr = np.zeros(nwin,dtype=np.int16)                  # number of correlations for each substack
            t_corr = dataS_t                                        # timestamp
            crap   = np.zeros(Nfft,dtype=np.complex64)
            for i in range(nwin):
                n_corr[i]= 1
                crap[:Nfft2] = corr[i,:]            #it was Nfft2
                crap[:Nfft2] = crap[:Nfft2]-np.mean(crap[:Nfft2])   # remove the mean in freq domain (spike at t=0). It was Nfft2
                crap[-(Nfft2)+1:] = np.flip(np.conj(crap[1:(Nfft2)]),axis=0)            #it was Nfft2
                crap[0]=complex(0,0)
                s_corr[i,:] = np.real(np.fft.ifftshift(scipy.fftpack.ifft(crap, Nfft, axis=0)))

            # remove abnormal data
            ampmax = np.max(s_corr,axis=1)
            tindx  = np.where( (ampmax<20*np.median(ampmax)) & (ampmax>0))[0]
            s_corr = s_corr[tindx,:]
            t_corr = t_corr[tindx]
            n_corr = n_corr[tindx]

        else:
            # get time information
            Ttotal = dataS_t[-1]-dataS_t[0]             # total duration of what we have now
            tstart = dataS_t[0]

            nstack = int(np.round(Ttotal/substack_len))
            ampmax = np.zeros(nstack,dtype=np.float32)
            s_corr = np.zeros(shape=(nstack,Nfft),dtype=np.float32)
            n_corr = np.zeros(nstack,dtype=np.int)
            t_corr = np.zeros(nstack,dtype=np.float)
            crap   = np.zeros(Nfft,dtype=np.complex64)

            for istack in range(nstack):
                # find the indexes of all of the windows that start or end within
                itime = np.where( (dataS_t >= tstart) & (dataS_t < tstart+substack_len) )[0]
                if len(itime)==0:tstart+=substack_len;continue

                crap[:Nfft2] = np.mean(corr[itime,:],axis=0)   # linear average of the correlation. It was Nfft2
                crap[:Nfft2] = crap[:Nfft2]-np.mean(crap[:Nfft2])   # remove the mean in freq domain (spike at t=0). It was Nfft2
                crap[-(Nfft2)+1:]=np.flip(np.conj(crap[1:(Nfft2)]),axis=0)            #it was Nfft2
                crap[0]=complex(0,0)
                s_corr[istack,:] = np.real(np.fft.ifftshift(scipy.fftpack.ifft(crap, Nfft, axis=0)))
                n_corr[istack] = len(itime)               # number of windows stacks
                t_corr[istack] = tstart                   # save the time stamps
                tstart += substack_len
                #print('correlation done and stacked at time %s' % str(t_corr[istack]))

            # remove abnormal data
            ampmax = np.max(s_corr,axis=1)
            tindx  = np.where( (ampmax<20*np.median(ampmax)) & (ampmax>0))[0]
            s_corr = s_corr[tindx,:]
            t_corr = t_corr[tindx]
            n_corr = n_corr[tindx]
#
#------ Not substack
    else:
        # average daily cross correlation functions
        ampmax = np.max(corr,axis=1)
        tindx  = np.where( (ampmax<20*np.median(ampmax)) & (ampmax>0))[0]
        n_corr = nwin
        s_corr = np.zeros(Nfft,dtype=np.float32)
        t_corr = dataS_t[0]
        crap   = np.zeros(Nfft,dtype=np.complex64)
        crap[:Nfft2] = np.mean(corr[tindx],axis=0)            #it was Nfft2
        crap[:Nfft2] = crap[:Nfft2]-np.mean(crap[:Nfft2],axis=0)            #it was Nfft2
        crap[-(Nfft2)+1:]=np.flip(np.conj(crap[1:(Nfft2)]),axis=0)            #it was Nfft2
        s_corr = np.real(np.fft.ifftshift(scipy.fftpack.ifft(crap, Nfft, axis=0)))

    # trim the CCFs in [-maxlag maxlag]
    t = np.arange(-Nfft2+1, Nfft2)*dt            #it was Nfft2
    ind = np.where(np.abs(t) <= maxlag)[0]
    if s_corr.ndim==1:
        s_corr = s_corr[ind]
    elif s_corr.ndim==2:
        s_corr = s_corr[:,ind]
#
#------  Returns
    return s_corr, t_corr, n_corr
#
# -------------- End of function   ---------------------


#
# ---------- extract the dispersion from the image ----------
"""
Takes the dispersion image from CWT as input, tracks the global maxinum on
    the wavelet spectrum amplitude and extract the sections with continous and high quality data
  <Args>
    amp:   2D amplitude matrix of the wavelet spectrum
    phase: 2D phase matrix of the wavelet spectrum
    per:   period vector for the 2D matrix
    vel:   vel vector of the 2D matrix
  <Returns>
    per:  central frequency of each wavelet scale with good data
    gv:   group velocity vector at each frequency
"""
def extract_dispersion(amp,per,vel):
#
    maxgap = 5
    nper = amp.shape[0]
    gv   = np.zeros(nper,dtype=np.float32)
    dvel = vel[1]-vel[0]
#
#------  find global maximum
    for ii in range(nper):
        maxvalue = np.max(amp[ii],axis=0)
        indx = list(amp[ii]).index(maxvalue)
        gv[ii] = vel[indx]
#
#------  check the continuous of the dispersion
    for ii in range(1,nper-15):
        # 15 is the minumum length needed for output
        for jj in range(15):
            if np.abs(gv[ii+jj]-gv[ii+1+jj])>maxgap*dvel:
                gv[ii] = 0
                break
#
#------  remove the bad ones
    indx = np.where(gv>0)[0]
#
#------  Returns
    return per[indx],gv[indx]
#
# -------------- End of function   ---------------------

print(f'\n>> Functions loaded.')


In [ ]:
"""
====================== Phone pairs ======================
"""
print(f'\n>> Choose pairs of the {len(gather)} using the polar angle of a rotating diameter.')
ent = input(f'\n<< Enter an angle step (rtn = 20 deg.):') or '20'
ent = int( ent.rstrip().split(' ')[0] )
matrix = u.pairs(ttb_loc, ent)
print(f'ph1|ph2|     distance(m)    |angle(deg.)|')
for row in matrix:
    print(" | ".join(map(str, row)))
#
#------ Choose a single pair
ent = input(f'\n<< Enter the 1st phone from above table:')
ent = int( ent.rstrip().split(' ')[0] )
i = next((i for i, row in enumerate(matrix) if row[0] == ent), -1)
print(f'\n>> Work with the pair [{matrix[i,0]}, {matrix[i,1]}], with a distance of {round(matrix[i,2],2)}m')
#
#------ Create an empty stream add the two traces
distance = matrix[i,2]
ntr1, ntr2 = matrix[i,:2] - 1
st = Stream()
st += gather[ntr1].copy()
st += gather[ntr2].copy()
print(f'Selected stream: {st}')
#
#------ Cut the trace to a window and taper it: 5% hann at both sides
ent = input(f' Enter t0 and t1 to cut: ')
ent = ent.rstrip().split(' ')
t0 = float(ent[0])
t1 = float(ent[1])
#
for tr in st:
    tr.trim(starttime=tr.stats.starttime + t0, endtime=tr.stats.starttime + t1)
    tr.taper(0.05)
#
print(f'Trimmed stream: {st}')
#
#------ plot
dummy = st[0].times(type="relative")         # time axis (s)
fNy = st[0].stats.sampling_rate / 2.         # Nyquist
for i in range(len(st)):
    FtrTplt = np.fft.rfft(st[i].data)[1:]        # Discard DC component
    if i == 0: freq = np.linspace(1, fNy, len(FtrTplt))   # Freq. axis (HZ). Discard f=0
    p.pltTrSp( dummy, st[i].data, freq, abs(FtrTplt),
              x1label='s', y1label='Ampl.', y1log=False, clr1 = 'k', 
              x2label='Hz', y2label='Spec.', y2log=True, clr2 = 'r' )

---
### Normalization and Spectral whitening

In [ ]:
"""
====================== Trace normalization and spectral whitening ======================
NB: tr0W below will be next_fast_len(len(FtrTplt)) greater than len(FtrTplt)
"""
#
#------- Work on first trace (source). Take the conjugate, so that the source spectrum is consistent with the receiver
tr0W = noise_processing(st[0].data, st[0].stats.delta, MTparam[3], MTparam[4], 
                     smooth_N = 100, time_norm = 'one_bit', freq_norm = 'rma')
tr0W = np.conjugate(tr0W)
#
#------- Plot whitened trace and its whitened spectrum
#        NB: trace will not be tapered as it comes from the Fourier inverse.
#--- Inverse fourier transform
trw = np.fft.ifft(tr0W)           # (tr0W[1])
#--- Discard small complex values left-overs after the inverse fourier transform.
trw = np.real(trw)
#--- Whitened trace has more points than the original one! Adjust time here.
dummy = st[0].times(type="relative")
dummy = np.linspace(dummy[0], dummy[-1], num = len(trw))
#--- The Nyquist is the original one
freq = np.linspace(0, fNy, len(tr0W))   # Freq. axis (HZ). f=0 is not discarded here
p.pltTrSp( dummy, trw, freq, abs(tr0W),
              x1label='s', y1label='Ampl.', y1log=False, clr1 = 'k', 
              x2label='Hz', y2label='Spec.', y2log=True, clr2 = 'r' )

In [ ]:
"""
====================== Trace normalization and spectral whitening ======================
"""
#
#------- Work on second trace(the receiver.
tr1W = noise_processing(st[1].data, st[1].stats.delta, MTparam[3], MTparam[4], 
                     smooth_N = 100, time_norm = 'one_bit', freq_norm = 'rma')
#
#------- Plot whitened trace and its whitened spectrum
#--- Inverse fourier transform
trw = np.fft.ifft(tr1W)           # (tr0W[1])
#--- Discard small complex values left-overs after the inverse fourier transform.
trw = np.real(trw)
p.pltTrSp( dummy, trw, freq, abs(tr1W),
              x1label='s', y1label='Ampl.', y1log=False, clr1 = 'k', 
              x2label='Hz', y2label='Spec.', y2log=True, clr2 = 'r' )

---
### Cross correlation

In [ ]:
"""
====================== Cross correlation ======================
"""
#
#------ Datetime object list to be used in correlation
dataS_t = [st[0].stats.starttime]
#
#------ Enter max lag
ent = st[0].stats.endtime - st[0].stats.starttime
print(f'\n>> Correlate trace {st[0].get_id()} with trace {st[1].get_id()}, with length={ent}s')
ent = input(f'\n<< Enter max lag time <={round(ent,1)} (rtn = {int(ent)}s):') or str(int(ent))
ent = ent.rstrip().split(' ')
maxlag = float(ent[0])
#corr| Tstamp| #segm.           source|receiver   NB: t_corr is a timestamp object
tdata, t_corr, n_corr = correlate(tr0W, tr1W, st[0].stats.delta, dataS_t,
                                  MTparam[3], MTparam[4], maxlag,
                                  method = 'xcorr')
#
#    dt = tr_source[0].stats.delta
#--- A sequential time for the x-correlation
tvec = np.linspace(-maxlag, maxlag, len(tdata))    #maxlag+dt
#
#------- plot correlation
fig, ax = plt.subplots(figsize=(12,5))
ax.plot(tvec,tdata)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Amplitude")
#ax.set_title('Cross-Correlation Function Between %s and %s'%(ssta,rsta))
plt.show()


---
### Dispersion Curves From Cross-Correlation Function
A frequency-time image is produced using the wavelet transform, and then converted into a velocity-period image. The group velocity curve is then selected from the ridge on the velocity-frequency image. In this example, we will take the mean of both the positive and negative lags of the cross-correlation function. This is known as the symmetrical component of the cross-correlation function. Modified from NoisePy.

The cross-correlations are symmetrized and turned into one-sided signals by averaging the positive and the negative lag parts. 

In [ ]:
import pprint
from scipy.interpolate import RegularGridInterpolator
#
# ---------- local script ----------
"""
Divides a vector at its center, flips the second part, and sums the two halves.
"""
def divSum(vector):
    n = len(vector)
    if n % 2 != 0:
        vector = np.append(vector, 0.)
        n += 1
#
    midpoint = int(n // 2)
    first  = vector[:midpoint]
    second = vector[midpoint:][::-1]  # Reverse the second half
#
    return first + second
# -------------- End of function
"""
====================== Dispersion Curve ======================
"""
#
#------ Set up parameters
fmin = MTparam[3]; pmax = 1. / fmin
fmax = MTparam[4]; pmin = 1. / fmax
dt = st[0].stats.delta
#--- wavelet transform parameters
dj, s0, J, wvn = [1/12, -1, -1, 'morlet']
#
#------ The symmetrical correlation
#--- number of points of the correalition
#    len(tvec)=801; tvec = [-8,..., 8]  
#npts = int(1/dt) * 2 * maxlag + 1              #npts = 511
#indx = int(npts // 2)                          #indx = 400
#--- len(tdata) = 511; tdata =[-5.8e-2,...,]
#--- data = 0.5 * tdata[indx:] + 0.5 * np.flip(tdata[:indx+1], axis=0) --- data = tdata[indx:]
#--- len(data) = 256; tdata =[-5.8e-2,...,]
#
ent = input(f'\n<< Acausal (a), causal (c), or sum of the two parts (s) of x-corr, len={len(tdata)})?') or str(int(ent))
ent = ent.rstrip().split(' ')
if ent[0] not in ['a', 'c', 's']: raise ValueError(f"'{ent}' not allowed")
indx = int(len(tdata) // 2)
if ent == 'a':
    data = tdata[:indx+1]
    tvec = tvec[:indx+1]
elif ent == 'c':
    data = tdata[indx+1:]
    tvec = tvec[indx+1:]
else:
    data = divSum(tdata) * 0.5
    tvec = np.linspace(0, maxlag, len(data))
#
print(f'>> Work with the {ent[0]}-part with len={len(data)}')
#
#------ wavelet transformation
cwt, sj, freq, coi, _, _ = pycwt.cwt(data, dt, dj, s0, J, wvn)
#
#------ filtering frequencies
if (fmax < np.max(freq)) | (fmax <= fmin):
    raise ValueError('Abort: frequency out of limits!')
freq_ind = np.where((freq >= fmin) & (freq <= fmax))[0]
cwt = cwt[freq_ind]
freq = freq[freq_ind]
#
#------ use amplitude of the cwt
period = 1 / freq
rcwt = np.abs(cwt) ** 2
#
print(f'\n>> Wavelet frequencies from {freq[0]} to {freq[-1]}Hz')
#
#------ interpolation to grids of freq-vel
#--- velocity range for or disperion analysis (m/s)
#    dt=0.02s and dist<30m -> vmax < 1000m/s
vmin, vmax = [150., 1000.]
vel  = np.linspace(vmin, vmax, len(period))
timevec = distance / vel
per  = np.linspace(pmin, pmax, len(period))
#

print(len(timevec))
print(len(period))
print(len(rcwt))


X, Y = np.meshgrid(period, vel)
Z = rcwt.reshape(len(X), len(Y))


# Create the interpolator
fc = RegularGridInterpolator((x, y), Z, method='linear', bounds_error=False, fill_value=np.nan)
rcwt_new = fc(per, vel)
#fc = scipy.interpolate.interp2d(distance / timevec, period, rcwt)
#
#------ normalization for each frequency
for ii in range(len(per)):
    rcwt_new[ii] /= np.max(rcwt_new[ii])


"""     Repository
how to separate the causal from the acausal parts of the cross-correlation?
print(indx)
print(len(tvec))
pprint.pprint(tvec)

"""


#
#------ Measure the dispersion curve from the velocity-period map
nper, gv = extract_dispersion(rcwt_new, per, vel)
#
#------ Find the first instance where the wavelength is greater than 1/3 of the inter-station distance
lmbda = gv * nper
bad_idx = np.where(lmbda > distance / 3)[0]
#
#------ Plot
fig, ax = plt.subplots(figsize=(12,6))
im = ax.imshow(np.transpose(rcwt_new),cmap='gnuplot2',extent=[per[0],per[-1],vel[0],vel[-1]],
           aspect='auto',origin='lower')
ax.plot(nper,gv,'b--', linewidth=4.0)
#--- Plot the first period where lambda is greater than the distance divided by 3
ax.plot([nper[bad_idx[0]], nper[bad_idx[0]]], [vel[0], vel[-1]],
        "-", color="black", linewidth=4.0)
ax.set_xlabel('Period [s]')
ax.set_ylabel('U [m/s]')
ax.set_title("Dispersion Curve")
cbar = plt.colorbar(im)
cbar.set_label("Normalized Amplitude")
plt.show()




---
## -----------------------------------------   Cut line

In [ ]:

#
print(f'>> Cut trace in segments to produce statistics. Enter parameters:')
print(f'    cc_len    -> Segment length (sec) to cut trace data')
print(f'    step      -> % overlapping (0, 1) between two contiguous sliding windows')
#
#------- I/P parameters
CUTparam = '3. 0.5'
# └─────>  [cc_len, step]
ent = str(CUTparam)
ent = input(f'\n Enter cc_len, step (dflt = {CUTparam})') or ent[:]
ent = ent.rstrip().split(' ')
cc_len, step = [float(dummy) for dummy in ent]

--------------------------------------------
#
#------ Cut trace 1 (source) into small segments and return statistics
#  sdev   |     mad   |timestamps|segmented data|
#                                             1st trace| segment(sec)|
trace_stdS, trace_madS, dataS_t, dataS = cut_trace(st[0], cc_len, step)
#
#------ 

-------------------------------------------------------------
dt   = st[0].stats.delta                  # sampling interval
fNy  = 1. / (2.0 * dt)                    # Nyquist frequency
-------------------------------------------------
smooth_N  = 100          # number of points to be smoothed for running-mean average (time-domain)
freq_norm   = 'rma'      # rma-> running mean average for frequency-domain normalization
time_norm   = 'one_bit'  # no-> no time-domain normalization; other options 'rma' for running-mean and 'one_bit'

------------------------------------------
#- time = st[ntr1].times(type="relative")
#
#------ Normalize and whiten
for tr in st:
#--- Taper the data with 10% Hanning
    tr.taper(type = 'hann', max_percentage = 0.1)
#--- Time normalization == 'one_bit' -> sign normalization
    tr = np.sign(tr)
#--- Whiten
#    tr = u.whiten(tr, MTparam[3], MTparam[4])
#    trw = whiten(tr, delta = dt, freqmin = MTparam[3], freqmax = MTparam[4], smooth_N  = 100)

---
## The direction-of-arrival (DOA). Choose an event with Beamforming
- A wavefront arrives at the surface at an angle $i$ with the vertical. The wave propagates toward the surface with a velocity $v_{c}=\frac{\Delta s}{\varDelta t}$, with a horizontal component $v_{h}=\frac{\Delta x}{\varDelta t}$.

- The horizontal slowness, $u_h$ is the inverse value of horizontal apparent velocity, $1/v_{h}$,
$$
u_{h}=\frac{1}{v_{h}}=\frac{\sin i}{\left|\mathbf{v}_{c}\right|},
$$
being related to: (a) the angle of incidence $i$, (b) the true velocity $v_c$ and (c) its azimuth with the North *toward* the epicenter; the **baz** ($\theta$).
$$
\boldsymbol{U}_0 = (\frac{\sin\theta}{v_{h}},\frac{\cos\theta}{v_{h}},\frac{1}{v_{h}\tan i})
                 = \frac{1}{v_{c}}(\sin i\sin\theta,\sin i\cos\theta,\cos i)
                 = u_{h}(\sin\theta,\cos\theta,\frac{1}{\tan i})
                 = \frac{1}{v_{c}}(\sin i\sin\theta,\sin i\cos\theta,\cos i).
$$

- The seismic signals at each sensor can be time-shifted and summed to enhance the S/N ratio by a factor of $\sqrt{N}$; the signals interfere constructively. The relative time shift of a given sensor $\boldsymbol{r}_{i},\,i=1,\ldots,N$, relatively to the center of the array is 
$$
\tau_{i}=\boldsymbol{r}_{i}.\boldsymbol{u}.
$$

- The beamforming for the array is,
$$
		b\left(t\right)=\frac{1}{N}\mathop{\sum_{i=1}^{N}s_{i}\left(t+
			\mathbf{r}_{i}\mathbf{\cdot u}\right)}=\frac{1}{N}\mathop{\sum_{i=1}^{N}s_{i}\left(t+\tau_{i}\right)}
$$

The ObsPy FK beamforming outputs the relative power, or semblance, and the absolute power, or FK power. Transforms the semblance $S$ to the Fisher or $F$-statistic as $F = (N-1) \frac{S}{1-S}$. $F\rightarrow1$ for white Gaussian noise, therefore if $F\neq1$ means that there is some signal.

<div style="text-align: center;">
<img src="./array1.png" width="600">
</div>



## Ambient Noise Cross-correlation
- Given two seismometers, $u_1$ and $u_2$, on the surface, will record ground motion as a function of time. Over long periods of time, the cross-correlation of ground motions is
$$C_{1,2}\left(\tau\right)=\int u_{1}\left(t\right)\,u_{2}\left(t+\tau\right)dt$$

- Data Preparation and inital processing
Prepare waveform data from each station separately to accentuate broad-band ambient noise.

In [ ]:
"""
====================== Trace correlation ======================
"""
print(f'\n>> Correlate trace {ntr1} with {st[0].stats.npts} with trace {ntr2} with {st[1].stats.npts} points')
ent = input(f'\n<< Enter max lag time (rtn = 2s):') or '2'
ent = float( ent.rstrip().split(' ')[0] )
max_lagtime = ent
#
max_shift_num = int(np.round(max_lagtime*st[0].stats.sampling_rate))
data1 = st[0].data
data2 = st[1].data
len1 = len(data1)
len2 = len(data2)
min_len = min(len1,len2)
#
cross_list = []
for shift_num in np.arange(-max_shift_num,max_shift_num+1,1):
    if shift_num<0:
        correlate_value = np.correlate(data1[:min_len+shift_num],data2[-shift_num:min_len])
        cross_list.append(correlate_value.ravel())
    else:
        correlate_value = np.correlate(data2[:min_len-shift_num],data1[shift_num:min_len])
        cross_list.append(correlate_value.ravel())
cross_list = np.array(cross_list)
cross_list = cross_list/np.max(cross_list)
#
fs_new = st[0].stats.sampling_rate
time = np.linspace(-max_lagtime,max_lagtime,int(2*max_lagtime*fs_new+1))
#-------- 
indexmax = np.argmax(cross_list)
travtime = time[indexmax]
print(f'\n>> Maximum lag = {travtime}s, corresponding to a velocity {np.round(distance/travtime, 2)}m/s')
#
plt.figure(figsize=(6, 2), dpi=180)
plt.plot(time,cross_list, 'k-')
#plt.plot(time, envelope, 'r-')
plt.axvline(travtime, 0.85, 1, color='b', lw=3)
plt.xlabel("Time (s)")
plt.ylabel("X-cor Coeff")
plt.xlim(-max_lagtime,max_lagtime)
plt.show();